In [2]:
import tensorflow as tf
import numpy as np


In [3]:
#Artificial dataset, lets say XOR
def xor():
    X = np.array([[0,1], [1,0], [1,1],[0,0]], dtype=float)
    Y = np.array([1,1,0,0], dtype=float).reshape((-1,1))
    return X,Y

def square_function():
    X = []
    Y = []
    for x in range(10):
        X.append(x)
        Y.append(x**2)
    return np.array(X,dtype=np.float32).reshape(-1,1),np.array(Y, dtype=np.float32).reshape(-1,1)
        

In [4]:
tf.nn.dropout?


# Simple feed forward neural network


In [5]:
#input

X,Y = square_function()

batch_size = 10

#Helper functions
def fully_connected(x, size, activation=tf.nn.relu):
    b = tf.Variable(tf.zeros([size]))
    weights = tf.Variable(tf.truncated_normal([x.get_shape()[1].value, size]))
    return activation(b + tf.matmul(x, weights))



INPUT_SHAPE = X.shape

inputLayer = tf.placeholder("float", [None,1])
y = tf.placeholder("float", [None,1])

#simple neural network to solve the xor problem

h1 = fully_connected(inputLayer, 20, activation=tf.nn.tanh)
print(h1.get_shape())

h2 = fully_connected(h1, 20)
print(h2.get_shape())

h3 = fully_connected(h2, 10)
print(h2.get_shape())

out = fully_connected(tf.nn.dropout(h2, 0.5), 1)
print(out.get_shape())

loss = tf.reduce_mean((out-y)**2)
tf.scalar_summary("loss", loss)


optimizer = tf.train.AdamOptimizer(0.1)
train_step = optimizer.minimize(loss)



(?, 20)
(?, 20)
(?, 20)
(?, 1)


# Run the session

In [7]:
init =  tf.initialize_all_variables()

num_epochs=1000
#merge = tf.merge_all_summaries()

with tf.Session() as sess:
    
    writer = tf.train.SummaryWriter("./summary", sess.graph)
    sess.run(init)
    #sess.run(merge)

    for epoch in range(num_epochs):
        for batch in range(X.shape[0]//batch_size):
            feed_dict = {
                inputLayer : X[batch*batch_size:batch*batch_size + batch_size],
                y : Y[batch*batch_size:batch*batch_size+batch_size]
            }
            _, lossVal = sess.run([train_step, loss], feed_dict=feed_dict)
        #writer.add_summary(merged, epoch)
        if(epoch%40 == 0):
            print("%d. Epoch Loss: %.4f "  %(epoch, lossVal))
        correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        
        

0. Epoch Loss: 1288.5931 
40. Epoch Loss: 355.7650 
80. Epoch Loss: 140.5761 
120. Epoch Loss: 336.6335 
160. Epoch Loss: 169.7115 
200. Epoch Loss: 87.6825 
240. Epoch Loss: 259.9925 
280. Epoch Loss: 35.8480 
320. Epoch Loss: 311.7480 


KeyboardInterrupt: 